In [ ]:
# import relevant modules
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
import imblearn
import sklearn
import os
import io
import pygraphviz
import pydotplus
import imageio
import itertools
#import xgboost as xgb

from scipy import misc
from scipy import stats
from IPython.display import Image
from sklearn import tree  
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import export_graphviz
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.externals.six import StringIO  
from sklearn.feature_selection import RFE
from sklearn.model_selection import train_test_split

# Ignore warnings
import warnings
warnings.filterwarnings('ignore')

# Settings
pd.set_option('display.max_columns', None)
np.set_printoptions(precision=3)
sns.set(style="darkgrid")
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12



In [ ]:
#Dataset loading
#FullDataframe = pd.read_csv("D:/DataSets/CIC-IDS-2017/MachineLearningCVE/Friday-WorkingHours-Afternoon-DDos.pcap_ISCX.csv") 

"""
from glob import glob
dfs = []
for file in sorted(glob('"D:/DataSets/CIC-IDS-2017/MachineLearningCVE/*.csv')):
    print(file)
    dfs.append(pd.read_csv(file))
FullDataframe = pd.concat(dfs)
"""

df6=pd.read_csv("C:/Users/QuatroPC/Desktop/Projeto4/DataSets/Canada 2017/MachineLearningCSV/MachineLearningCVE/Thursday-WorkingHours-Morning-WebAttacks.pcap_ISCX.csv")
#df7=pd.read_csv("C:/Users/QuatroPC/Desktop/Projeto4/DataSets/Canada 2017/MachineLearningCSV/MachineLearningCVE/Tuesday-WorkingHours.pcap_ISCX.csv")
#df8=pd.read_csv("C:/Users/QuatroPC/Desktop/Projeto4/DataSets/Canada 2017/MachineLearningCSV/MachineLearningCVE/Wednesday-workingHours.pcap_ISCX.csv")

FullDataframe = pd.concat([df6])
print("Original version has {} rows & {} columns".format(FullDataframe.shape[0],FullDataframe.shape[1]))

FullDataframe = FullDataframe.drop_duplicates()
print("No-duplicates version has {} rows & {} columns".format(FullDataframe.shape[0],FullDataframe.shape[1]))

In [ ]:
#FullDataframe.groupby('Label').first()
FullDataframe[' Label'].value_counts()

In [ ]:
print(FullDataframe.head(4))

In [ ]:
FullDataframe.info()

In [ ]:
FullDataframe.describe()

In [ ]:
FullDataframe

In [ ]:
train, test = train_test_split(FullDataframe, test_size = 0.20, random_state=10)

train[' Label'].value_counts()

In [ ]:
test[' Label'].value_counts()

In [ ]:
train_y = train[' Label']
train_x = train.drop([' Label'], axis=1)
#train_y

In [ ]:
train_x2 = train_x
train_x = train_x.values.astype(np.long)
np.nan_to_num(train_x)

#train_x2

In [ ]:
# create and Random Forest model and most important features
DTC_Classifier = DecisionTreeClassifier()
DTC_Classifier.fit(train_x, train_y);

score = np.round(DTC_Classifier.feature_importances_,3)
importances = pd.DataFrame({'feature':train_x2.columns,'importance':score})
importances = importances.sort_values('importance',ascending=False).set_index('feature')
# plot importances
plt.rcParams['figure.figsize'] = (11, 4)
importances.plot.bar();

In [ ]:
# create the RFE model and select 15 attributes
KN_Classifier_RFE = DecisionTreeClassifier()
train_test = train

rfe = RFE(KN_Classifier_RFE, n_features_to_select=15)
rfe = rfe.fit(train_x, train_y)

In [ ]:
# summarize the selection of the attributes
feature_map = [(i, v) for i, v in itertools.zip_longest(rfe.get_support(), train_x2.columns)]
selected_features = [v for i, v in feature_map if i==True]

selected_features

In [ ]:
from sklearn.svm import SVC 
from sklearn.naive_bayes import BernoulliNB 
from sklearn import tree
from sklearn.model_selection import cross_val_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression

# Train KNeighborsClassifier Model
KNN_Classifier = KNeighborsClassifier(n_jobs=-1)
KNN_Classifier.fit(train_x, train_y); 

# Train LogisticRegression Model
LGR_Classifier = LogisticRegression(n_jobs=-1, random_state=42)
LGR_Classifier.fit(train_x, train_y);

# Train Gaussian Naive Baye Model
BNB_Classifier = BernoulliNB()
BNB_Classifier.fit(train_x, train_y)
            
# Train Decision Tree Model
DTC_Classifier = tree.DecisionTreeClassifier(criterion='entropy', random_state=42)
DTC_Classifier.fit(train_x, train_y)

# Already trained Random Forest Model -> RFC_Classifier

In [ ]:
from sklearn import metrics

models = []
models.append(('Naive_Baye', BNB_Classifier))
models.append(('Decision_Tree', DTC_Classifier))
models.append(('KNeighbors', KNN_Classifier))
models.append(('Logistic_Regression', LGR_Classifier))

for i, v in models:
    scores = cross_val_score(v, train_x, train_y, cv=100)
    accuracy = metrics.accuracy_score(train_y, v.predict(train_x))
    confusion_matrix = metrics.confusion_matrix(train_y, v.predict(train_x))
    classification = metrics.classification_report(train_y, v.predict(train_x))
    print()
    print('============================== {} Model Evaluation =============================='.format(i))
    print()
    print ("Cross Validation Mean Score:" "\n", scores.mean())
    print()
    print ("Model Accuracy:" "\n", accuracy)
    print()
    print("Confusion matrix:" "\n", confusion_matrix)
    print()
    print("Classification report:" "\n", classification) 
    print()


In [ ]:
test_y = test[' Label']
test_x = test.drop([' Label'], axis=1)
test_y
test_x

In [ ]:
test_x = test_x.values.astype(np.long)
np.nan_to_num(test_x)

In [ ]:
for i, v in models:
    accuracy = metrics.accuracy_score(test_y, v.predict(test_x))
    confusion_matrix = metrics.confusion_matrix(test_y, v.predict(test_x))
    classification = metrics.classification_report(test_y, v.predict(test_x))
    print()
    print('============================== {} Model Test Results =============================='.format(i))
    print()
    print ("Model Accuracy:" "\n", accuracy)
    print()
    print("Confusion matrix:" "\n", confusion_matrix)
    print()
    print("Classification report:" "\n", classification) 
    print()      

In [ ]:
pred_knn = KNN_Classifier.predict(test_x)
pred_NB = BNB_Classifier.predict(test_x)
pred_log = LGR_Classifier.predict(test_x)
pred_dt = DTC_Classifier.predict(test_x)

In [ ]:
pred_NB

In [ ]:
pred_log

In [ ]:
pred_dt

In [ ]:
pred_knn

In [ ]:
import pickle

for i, v in models:
        print(i)
        filename = 'model_'+i+'.sav'
        pickle.dump(v, open(filename, 'wb'))